In [2]:
import pandas as pd 
import numpy as np
from os import listdir

In [10]:
data_read_path = "..//Data//"
compound_list_path = data_read_path + "Compound Constraints//"
spectra_path = data_read_path + "Deconvoluted Spectra//"

#read in data
spectra_files = listdir(spectra_path)
bound_df = pd.read_excel(spectra_path+spectra_files[0])

compounds_files = listdir(compound_list_path)
compounds = pd.read_excel(compound_list_path+compounds_files[0])